In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
lis = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        lis.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import PIL.Image as image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
from plotly.offline import iplot, plot, init_notebook_mode
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import tqdm
import warnings

In [ ]:
TRAIN_IMGS = '/kaggle/input/cassava-leaf-disease-classification/train_images/'
TEST_IMGS = '/kaggle/input/cassava-leaf-disease-classification/test_images/'

In [ ]:
warnings.filterwarnings('ignore')
init_notebook_mode('connected')
plt.rcParams['figure.figsize'] = [12,6]
tqdm.tqdm.pandas()

In [ ]:
train_df = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')

In [ ]:
train_df['label_names']=np.select(choicelist=["Cassava Bacterial Blight (CBB)",
                                              "Cassava Brown Streak Disease (CBSD)", 
                                              "Cassava Green Mottle (CGM)", 
                                              "Cassava Mosaic Disease (CMD)",
                                              "Healthy"],
                                 condlist=[train_df['label']==0,
                                           train_df['label']==1,
                                           train_df['label']==2,
                                           train_df['label']==3,
                                           train_df['label']==4])

In [ ]:
fig = plt.subplot(1, 2, 1).figure
# fig.set_figheight(10)
# fig.set_figwidth(25)
sns.countplot('label_names', data = train_df)
plt.title('Count of label names', fontdict={'size':20, 'color':'white'})
plt.tick_params(labelrotation=90, labelsize = 15, axis = 'x')
plt.tick_params(labelcolor='white', axis = 'y')


# ax.set_figheight(10)

plt.subplot(1, 2, 2)
sns.countplot('label', data = train_df)
plt.title('Count of label', fontdict = {'size':20, 'color':'white'})
plt.tick_params(labelsize = 15, labelcolor='white', axis = 'x')

In [ ]:
def display_images(label, rows=2, cols = 2, image = False):
    images = train_df[train_df['label'] == label]['image_id'].sample(rows*cols).values
    fig, ax = plt.subplots(nrows = rows, ncols = cols, figsize= (15, 10))
    ax = ax.flatten()
    for i in range(len(images)):
        quick = cv2.imread(TRAIN_IMGS+images[i])
        quick = cv2.cvtColor(quick, cv2.COLOR_BGR2RGB)
        ax[i].imshow(quick)
        ax[i].set_title(train_df[train_df['label']==label]['label_names'].head(1).values[0])

In [ ]:
display_images(0)

In [ ]:
display_images(1)

In [ ]:
display_images(2)

In [ ]:
display_images(3)

In [ ]:
display_images(4)

In [ ]:
def color_channels(label):
    img_details = train_df[train_df['label']==label].sample(1).values
    quick = cv2.imread(TRAIN_IMGS+img_details[:, 0][0])
    quick = cv2.cvtColor(quick, cv2.COLOR_BGR2RGB)
    fig = make_subplots(1, 2)
    red = go.Histogram({'x': cv2.calcHist(quick, [0], None, [255], [0, 255]).reshape(-1), 'text':'Red', 'marker':{'color':'red'}, 'name':'Red', 'xbins':{'size':1}})
    green = go.Histogram({'x':cv2.calcHist(quick, [1], None, [255], [0, 255]).reshape(-1), 'text':'Green', 'marker':{'color':'green'}, 'name':'Green', 'xbins':{'size':1}})
    blue = go.Histogram({'x':cv2.calcHist(quick, [2], None, [255], [0, 255]).reshape(-1), 'text':'Blue', 'marker':{'color':'blue'}, 'name':'Blue', 'xbins':{'size':1}})
    fig.add_trace(red, row =1, col=2)
    fig.add_trace(green, row= 1, col=2)
    fig.add_trace(blue, row = 1, col=2)
    fig.add_trace(go.Image({'name' : img_details[:, 2][0]}, z = quick ))
    layout = {'title':'Color channel Histogram', 'barmode':'stack', 'template':'simple_white'}
    fig.update_layout(layout)
    # fig = go.Figure(data = data, layout=layout)
    iplot(fig)

In [ ]:
color_channels(0)

In [ ]:
color_channels(1)

In [ ]:
color_channels(2)

In [ ]:
color_channels(3)

In [ ]:
color_channels(4)

In [ ]:
dat = train_df.head(1)
quick = cv2.imread(TRAIN_IMGS+dat.values[:, 0][0])
quick = cv2.cvtColor(quick, cv2.COLOR_BGR2RGB)
fig = make_subplots(rows=2, cols = 2)
fig2 = ff.create_distplot([quick[:, :, 0][0]], ['Red'], colors=['red'])
fig3  = ff.create_distplot([quick[:, :, 1][0]], ['Green'], colors = ['green'])
fig4 = ff.create_distplot([quick[:, :, 2][0]], ['Blue'], colors = ['blue'])
fig.add_trace(go.Histogram(fig2['data'][0]), row = 1, col = 1)
fig.add_trace(go.Scatter(fig2['data'][1]), row = 1, col = 1)
fig.add_trace(go.Histogram(fig3['data'][0]), row=1, col = 2)
fig.add_trace(go.Scatter(fig3['data'][1]), row = 1, col=2)
fig.add_trace(go.Histogram(fig4['data'][0]), row = 2, col = 1)
fig.add_trace(go.Scatter(fig4['data'][1]), row = 2, col = 1)
layout = dict(title = "Distribution of Color channel values in %s." %dat.values[:, 0][0])
fig.update_layout(layout)
fig.show()

In [ ]:
# test=ImageDataGenerator(rescale=1/255).flow_from_dataframe(train_df.head(100),
#                                                            TRAIN_IMGS, x_col='image_id',
#                                                            y_col = 'label', class_mode='raw',
#                                                            shuffle=False, batch_size=100, target_size = (600, 800))
# train_images = test[0][0]

In [ ]:
def load_image(image_id):
    image = cv2.imread(TRAIN_IMGS + image_id)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
train_images = train_df["image_id"][:100].progress_apply(load_image)

In [ ]:
red_values = [np.mean(train_images[idx][:, :, 0]) for idx in range(len(train_images))]
green_values = [np.mean(train_images[idx][:, :, 1]) for idx in range(len(train_images))]
blue_values = [np.mean(train_images[idx][:, :, 2]) for idx in range(len(train_images))]
values = [np.mean(train_images[idx]) for idx in range(len(train_images))]

In [ ]:
fig = make_subplots(rows=2, cols = 2)
# trace1 = dict(x = values,  = 'Channels', text = 'Channels', marker = dict(color='purple'))
vals = ff.create_distplot([values], group_labels=['Channels'], colors=['purple'])
reds  = ff.create_distplot([red_values], group_labels=['Red Values'], colors = ['red'])
greens = ff.create_distplot([green_values], group_labels=['Green Values'], colors = ['green'])
blues = ff.create_distplot([blue_values], group_labels = ['Blue Values'], colors = ['blue'])

fig.add_trace(vals['data'][0], row=1, col = 1)
fig.add_trace(vals['data'][1], row = 1, col = 1)
fig.add_trace(reds['data'][0], row = 1, col = 2)
fig.add_trace(reds['data'][1], row = 1, col = 2)
fig.add_trace(greens['data'][0], row=2, col = 1)
fig.add_trace(greens['data'][1], row = 2, col = 1)
fig.add_trace(blues['data'][0], row = 2, col= 2)
fig.add_trace(blues['data'][1], row = 2, col = 2)

fig.update_traces({'marker':{'line':{'width':0.3}}})

In [ ]:
trace1 = dict(y = red_values, type = 'box', name = 'Red', marker = dict(color = 'red'), text = 'Red values')
trace2 = dict(y = green_values, type = 'box', name = 'Green', marker = dict(color = 'green'), text = 'Green values')
trace3 = dict(y = blue_values, type = 'box', name = 'Blue', marker = dict(color = 'blue'), text = 'Blue values')
fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.add_trace(trace3)
layout = dict(title = 'Color values box plot')
fig.update_layout(layout)

iplot(fig)

# Blurring

In [ ]:
fig = plt.subplot(1, 2, 1)
fig.figure.set_figheight(30)
fig.figure.set_figwidth(30)
plt.imshow(train_images[0])
plt.title('Normal Image', fontdict={'size':30})

kernel = np.ones((5, 5), dtype = np.float32)/22
img = cv2.filter2D(train_images[0], -1,kernel)
plt.subplot(1, 2, 2)
plt.imshow(img)
plt.title('Image blurred using filter2D with kernel', fontdict={'size':30})

In [ ]:
fig = plt.subplot(1, 2, 1)
fig.figure.set_figheight(30)
fig.figure.set_figwidth(30)
plt.imshow(train_images[99])
plt.title('Normal Image', fontdict={'size':30})


plt.subplot(1, 2, 2)
plt.imshow(cv2.blur(train_images[99], (5, 5)))
plt.title('Image blurred using blur', fontdict = {'size':30})

In [ ]:
fig = plt.subplot(1, 2, 1)
fig.figure.set_figheight(30)
fig.figure.set_figwidth(30)
plt.imshow(train_images[27])
plt.title('Normal Image', fontdict={'size':30})

plt.subplot(1, 2, 2)
plt.imshow(cv2.GaussianBlur(train_images[27], (5, 5), 10))
plt.title('Image blurred using GaussianBlur', fontdict = {'size':30})

In [ ]:
fig = plt.subplot(1, 2, 1)
fig.figure.set_figheight(30)
fig.figure.set_figwidth(30)
plt.imshow(train_images[69])
plt.title('Normal Image', fontdict = {'size':20})

plt.subplot(1, 2, 2)
plt.imshow(cv2.medianBlur(train_images[69], 5))
plt.title('Image blurred using medianBlurr', fontdict = {'size':20})

In [ ]:
fig = plt.subplot(1, 2, 1)
fig.figure.set_figheight(30)
fig.figure.set_figwidth(30)
plt.imshow(train_images[20])
plt.title('Normal Image', fontdict = {'size':20})

plt.subplot(1, 2, 2)
plt.imshow(cv2.bilateralFilter(train_images[20], 11, 75, 75))
plt.title('Image blurred using bilateralFilter');

# Blending

In [ ]:
sticker = cv2.resize(train_images[10], (250, 250))

In [ ]:
plt.figure(figsize = (12, 5))
plt.imshow(train_images[11])

In [ ]:
#Overlaying images of different sizes
copy = train_images[11].copy()
copy[350:, 550:] = sticker

In [ ]:
plt.figure(figsize = (12, 6))
plt.imshow(copy)

In [ ]:
#using addFilter
print(train_images[10].shape)
print(train_images[11].shape)

plt.figure(figsize = (10, 6))
plt.imshow(cv2.addWeighted(train_images[10], 0.6, train_images[11], 0.6, 0.3))

In [ ]:
#Blending images of different sizes
sticker = cv2.imread('/kaggle/input/opencv-practice-zip/computer-vision-with-python/Computer-Vision-with-Python/DATA/watermark_no_copy.png')
sticker = cv2.cvtColor(sticker, cv2.COLOR_BGR2RGB) 
plt.imshow(sticker)
print(sticker.shape)

sticker = cv2.resize(sticker, (250, 250))

In [ ]:
#Blending images of different sizes
gray_sticker = cv2.cvtColor(sticker, cv2.COLOR_RGB2GRAY)
mask_inv = cv2.bitwise_not(gray_sticker)
white_background = np.full(sticker.shape, 255, dtype=np.uint8)
fg = cv2.bitwise_or(sticker, sticker, mask = mask_inv)
roi = train_images[12][:250, :250]
train_images[12][:250, :250]= cv2.bitwise_or(roi, fg)

plt.figure(figsize=(12, 6))
plt.imshow(train_images[12])

In [ ]:
#Gradiets
sobel_x = cv2.Sobel(cv2.cvtColor(train_images[11], cv2.COLOR_RGB2GRAY), None, 1, 0, (7, 7))
sobel_y = cv2.Sobel(cv2.cvtColor(train_images[11], cv2.COLOR_RGB2GRAY), None, 0, 1, (7, 7))

In [ ]:
plt.figure(figsize = (10, 6))
plt.imshow(cv2.addWeighted(sobel_x, 0.6, sobel_y, 0.6, 0.5), cmap = 'gray')

# Thresholding

In [ ]:
fig = plt.subplot(1, 2, 1)
fig.figure.set_figheight(30)
fig.figure.set_figwidth(30)
plt.imshow(train_images[69])
plt.title('Normal Image', fontdict = {'size':20})


plt.subplot(1, 2, 2)
ret, thresh_img = cv2.threshold(cv2.cvtColor(train_images[69], cv2.COLOR_RGB2GRAY), 120, 255, cv2.THRESH_BINARY)
plt.imshow(thresh_img, cmap = 'gray')
plt.title('Thresholded image using THRESH_BINARY', fontdict = {'size':20})

In [ ]:
fig = plt.subplot(1, 2, 1)
fig.figure.set_figheight(30)
fig.figure.set_figwidth(30)
plt.imshow(train_images[69])
plt.title('Normal Image', fontdict = {'size':20})


plt.subplot(1, 2, 2)
ret, thresh_img = cv2.threshold(cv2.cvtColor(train_images[69], cv2.COLOR_RGB2GRAY), 120, 255, cv2.THRESH_BINARY_INV)
plt.imshow(thresh_img, cmap = 'gray')
plt.title('Thresholded image using THRESH_BINARY_INV', fontdict = {'size':20})

In [ ]:
fig = plt.subplot(1, 2, 1)
fig.figure.set_figheight(30)
fig.figure.set_figwidth(30)
plt.imshow(train_images[69])
plt.title('Normal Image', fontdict = {'size':20})


plt.subplot(1, 2, 2)
ret, thresh_img = cv2.threshold(cv2.cvtColor(train_images[69], cv2.COLOR_RGB2GRAY), 120, 255, cv2.THRESH_TOZERO)
plt.imshow(thresh_img, cmap = 'gray')
plt.title('Thresholded image using THRESH_TOZERO', fontdict = {'size':20})

In [ ]:
fig = plt.subplot(1, 2, 1)
fig.figure.set_figheight(30)
fig.figure.set_figwidth(30)
plt.imshow(train_images[69])
plt.title('Normal Image', fontdict = {'size':20})


plt.subplot(1, 2, 2)
ret, thresh_img = cv2.threshold(cv2.cvtColor(train_images[69], cv2.COLOR_RGB2GRAY), 120, 255, cv2.THRESH_TOZERO_INV)
plt.imshow(thresh_img, cmap = 'gray')
plt.title('Thresholded image using THRESH_TOZERO_INV', fontdict = {'size':20})

In [ ]:
fig = plt.subplot(1, 2, 1)
fig.figure.set_figheight(30)
fig.figure.set_figwidth(30)
plt.imshow(train_images[69])
plt.title('Normal Image', fontdict = {'size':20})


plt.subplot(1, 2, 2)
ret, thresh_img = cv2.threshold(cv2.cvtColor(train_images[69], cv2.COLOR_RGB2GRAY), 120, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
plt.imshow(thresh_img, cmap = 'gray')
plt.title('Thresholded image using THRESH_TOZERO', fontdict = {'size':20})

# Morphology

In [ ]:
#Create a black Blank image
blank_img = np.zeros((1080, 1920))
#Add text ABCDE to blank image
cv2.putText(blank_img, "Daimond Hands", (350, 600), cv2.FONT_HERSHEY_SIMPLEX,5, (255, 255, 255), 25, cv2.LINE_AA)

plt.imshow(blank_img, cmap = 'gray')

In [ ]:
#Erode foreground
kernel = np.ones((5, 5), np.uint8)
plt.imshow(cv2.erode(blank_img, kernel, iterations = 3), cmap = 'gray')

In [ ]:
#Remove White Noise
white_noise = np.random.randint(0, 2, (1080, 1920))
white_noise = white_noise*255
img = blank_img.copy()
noise_img = img + white_noise


plt.subplot(1, 2, 1)
plt.imshow(noise_img, cmap = 'gray')
plt.title('Noisy Image')


plt.subplot(1, 2, 2)
plt.imshow(cv2.morphologyEx(noise_img.astype('uint8'), cv2.MORPH_OPEN, kernel), cmap = 'gray')
plt.title('De-Noised Image')

In [ ]:
black_noise = np.random.randint(0, 2, (1080, 1920))
black_noise = black_noise*-255
noise_img = img + black_noise
noise_img[noise_img==-255] = 0

plt.subplot(1, 2, 1)
plt.imshow(noise_img, cmap = 'gray')
plt.title('Noisy image')

plt.subplot(1, 2, 2)
plt.imshow(cv2.morphologyEx(noise_img, cv2.MORPH_CLOSE, kernel), cmap = 'gray')
plt.title('De-Noised image');

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(img, cmap = 'gray')
plt.title('Noisy, image')

plt.subplot(1, 2, 2)
plt.imshow(cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel), cmap = 'gray')
plt.title('Hollowed image')

# Corner Detection

In [ ]:
chess = cv2.imread('/kaggle/input/opencv-practice-zip/computer-vision-with-python/Computer-Vision-with-Python/DATA/flat_chessboard.png')
chess = cv2.cvtColor(chess, cv2.COLOR_BGR2RGB)
plt.subplot(1, 2, 1)
plt.imshow(chess)

ret, corners = cv2.findChessboardCorners(chess, (7, 7))
plt.subplot(1, 2, 2)
copy = chess.copy()
plt.imshow(cv2.drawChessboardCorners(copy, (7, 7), corners, ret));

In [ ]:
#Finding corners of corner harris algorithm
dst = cv2.cornerHarris(cv2.cvtColor(train_images[12], cv2.COLOR_RGB2GRAY), 3, 3, 0.01)

dst = cv2.dilate(dst, None)
copy = train_images[12].copy()
copy[dst>0.06*dst.max()] = [255, 0, 0]

plt.imshow(copy);

In [ ]:
#Finding corners using corner harris algorithm
dst = cv2.cornerHarris(cv2.cvtColor(chess, cv2.COLOR_RGB2GRAY), 3, 3, 0.1)

dst = cv2.dilate(dst, None)

copy = chess.copy()

copy[dst>0.06*dst.max()] = [255, 0, 0]

plt.imshow(copy);

In [ ]:
#Find corners using goodFeaturesToTrack(Shi Tomasi algorithm)

corners = cv2.goodFeaturesToTrack(cv2.cvtColor(chess, cv2.COLOR_RGB2GRAY), 50, 0.01, 4, 10, None)

corners = np.int0(corners)
copy = chess.copy()
for i in corners:
    x, y = i.ravel()
    cv2.circle(copy, (x, y), 3, 255, -1)
plt.imshow(copy);

In [ ]:
#Find corners using goodFeaturesToTrack(Shi Tomasi algorithm)

copy = train_images[42].copy()
corners = cv2.goodFeaturesToTrack(cv2.cvtColor(copy, cv2.COLOR_RGB2GRAY), 30, 0.06, 3)

corners = np.int0(corners)

for i in corners:
    x, y = i.ravel()
    cv2.circle(copy, (x, y), 10, 255, -1)
    
plt.imshow(copy)

# Edge Detection

In [ ]:
#Canny Edge detection
fig = plt.subplot(1, 2, 1)
# fig.figure.set_figheight(50)
# fig.figure.set_figwidth(50)
plt.imshow(train_images[0])
plt.subplot(1, 2, 2)
plt.imshow(cv2.Canny(train_images[0], 100, 200), cmap= 'gray')

In [ ]:
#Canny Edge detection after blurring
fig = plt.subplot(1, 3, 1)
fig.figure.set_figheight(20)
fig.figure.set_figwidth(20)
plt.imshow(train_images[0])
plt.title('Normal image')

copy = train_images[0].copy()

blurred = cv2.GaussianBlur(copy, (7, 7), 10)
plt.subplot(1, 3, 2)
plt.imshow(blurred)
plt.title('Blurred image')

plt.subplot(1, 3, 3)
plt.imshow(cv2.Canny(blurred, 100, 200))
plt.title('Edges after blurring')

In [ ]:
#Grid detection
dots = cv2.imread('/kaggle/input/opencv-practice-zip/computer-vision-with-python/Computer-Vision-with-Python/DATA/dot_grid.png')
# dots = cv2.cvtColor(dots, cv2.COLOR_BGR2RGB)
plt.imshow(dots)

In [ ]:
#Grid detection
ret, corners = cv2.findCirclesGrid(dots, (10, 10), cv2.CALIB_CB_SYMMETRIC_GRID)
copy = dots.copy()
plt.imshow(cv2.drawChessboardCorners(copy, (10, 10), corners, ret))

# Contour detection

In [ ]:
#Contour detection using findContours
img = cv2.imread('/kaggle/input/opencv-practice-zip/Computer-Vision-with-Python/DATA/internal_external.png', 0)
plt.imshow(img, cmap = 'gray')

In [ ]:
#External Contours
contours, hierarchy = cv2.findContours(img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

external_countours = np.zeros(img.shape)

for i in range(len(contours)):
    if hierarchy[0][i][3] == -1:
        cv2.drawContours(external_countours, contours, i, 255, 5)
        
plt.imshow(external_countours, cmap = 'gray')

In [ ]:
#Internal Contours
internal_contours = np.zeros(img.shape)
for i in range(len(contours)):
    if hierarchy[0][i][3] != -1:
        cv2.drawContours(internal_contours, contours, i, 255, -1)
        
plt.imshow(internal_contours, cmap= 'gray')

In [ ]:
contours, hierarchy = cv2.findContours(cv2.cvtColor(train_images[69], cv2.COLOR_RGB2GRAY), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

external_countours = np.zeros(train_images[69].shape)

for i in range(len(contours)):
    if hierarchy[0][i][3] == -1:
        cv2.drawContours(external_countours, contours, i, 255, 5)
        
plt.imshow(external_countours)

# Feature Matching

In [ ]:
img1 = train_images[10]
img2 = train_images[14]

In [ ]:
plt.imshow(img1)

In [ ]:
plt.imshow(img2)

In [ ]:
#Brute Force detection with ORB descreptors
orb = cv2.ORB_create()

kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)

matches = bf.match(des1, des2)

matches = sorted(matches , key = lambda x:x.distance)

img1_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:10], None, flags = 2)

In [ ]:
plt.imshow(img1_matches)
plt.title('ORB feature detection')

In [ ]:
#Brute Force detection with SIFT Descriptors and Ratio Test
sift = cv2.SIFT_create()

kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

bf = cv2.BFMatcher()

matches = bf.knnMatch(des1, des2, k = 2)

good = []

for match1, match2 in matches:
    if match1.distance < 0.75*match2.distance:
        good.append([match1])
sift_matches = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, flags=2)

In [ ]:
plt.imshow(sift_matches)
plt.title('SIFT Descriptors')

In [ ]:
sift = cv2.SIFT_create()

kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(des1, des2, k = 2)

good = []

for i, (match1, match2) in enumerate(matches):
    if match1.distance < 0.7* match2.distance:
        good.append(match1)
    
flann_matches = cv2.drawMatches(img1, kp1, img2, kp2, good, None, flags=2)

plt.imshow(flann_matches)